# 3.1 Simulated Annealing

Install required packages / libraries (or update).

In [ ]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 2.1/2.1 MB 2.7 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\ProgramData\miniconda3\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 1.4/1.4 MB 1.1 MB/s eta 0:00:00


In [30]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 13.2/13.2 MB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 8.5/8.5 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 8.7/8.7 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 994.8/994.8 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 6.8/6.8 MB 557.8 kB/s eta 0:00:00
     ------------------------------------ 104.1/104.1 kB 665.7 kB/s eta 0:00:00
     -------------------------------------- 55.8/55.8 kB 584.1 kB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 796.6 kB/s eta 0:00:00
     ------------------------------------ 508.0/508.0 kB 817.3 kB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 M

## 3.1.1 Initialization

Begin the program by loading the dataset.

In [31]:
import pandas as pd

# Load the CSV files
gas_stations_df = pd.read_csv('dataset/gas_stations.csv')
distances_df = pd.read_csv('dataset/x_y_distances.csv')

# Preview the data
gas_stations_df.head(), distances_df.head()

OSError: [WinError 193] %1 is not a valid Win32 application

### 3.1.2 Import Necessary Libraries

This probabilistic technique is used for approximating the global optimum of a given function. It will find the shortest route across the gas stations, beginning from DISPERINDAG Surabaya.

In [ ]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

### 3.1.3 Execution

This section defines the distances, starting point, base temperature, and cooling schedule.

In [ ]:
def simulated_annealing(distances, start_idx, temperature=1000, cooling_rate=0.003, min_temp=1):
    num_nodes = len(distances)
    
    # Generate initial solution (random path)
    current_solution = list(range(num_nodes))
    random.shuffle(current_solution)
    current_solution.insert(0, start_idx)  # Start at DISPERINDAG
    
    current_distance = calculate_total_distance(current_solution, distances)
    best_solution = current_solution.copy()
    best_distance = current_distance
    
    # Annealing process
    while temperature > min_temp:
        # Generate new solution by swapping two nodes
        new_solution = current_solution.copy()
        i, j = random.sample(range(1, num_nodes), 2)  # avoid swapping the start node
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        
        new_distance = calculate_total_distance(new_solution, distances)
        
        # Decide whether to accept the new solution
        if accept_solution(current_distance, new_distance, temperature):
            current_solution = new_solution
            current_distance = new_distance
        
        # Update the best solution if found a new best
        if current_distance < best_distance:
            best_solution = current_solution.copy()
            best_distance = current_distance
        
        # Cool the system
        temperature *= 1 - cooling_rate
    
    return best_solution, best_distance


### 3.1.4 Travel Distance

This section will calculate the total distance traveled according to the number and complexity of nodes.

In [ ]:
def calculate_total_distance(solution, distances):
    total_distance = 0
    for i in range(len(solution) - 1):
        total_distance += distances.loc[solution[i], solution[i + 1]]
    return total_distance

### 3.1.5 Choose Best Route

This section will pick the best / optimal solution according to the thermal expansion and cooling process in the region.

In [ ]:
def accept_solution(current_distance, new_distance, temperature):
    if new_distance < current_distance:
        return True
    else:
        # Accept the new solution with a certain probability
        return random.random() < math.exp((current_distance - new_distance) / temperature)

# 3.2 Lovebird Algorithm

## 3.2.1 Define algorithm

This section provides the necessary features for the algorithm through the number of nodes and agents iterating through them.

In [ ]:
def lovebird_algorithm(distances, start_idx, num_agents=10, max_iterations=100):
    num_nodes = len(distances)
    
    # Initialize agents randomly
    agents = [list(range(num_nodes)) for _ in range(num_agents)]
    for agent in agents:
        random.shuffle(agent)
        agent.insert(0, start_idx)  # Start at DISPERINDAG

    best_agent = None
    best_distance = float('inf')

    for iteration in range(max_iterations):
        for agent in agents:
            # Calculate the total distance for this agent's solution
            current_distance = calculate_total_distance(agent, distances)
            
            # Update the best solution if this agent is better
            if current_distance < best_distance:
                best_distance = current_distance
                best_agent = agent
        
            # Move agents toward the best solution
            for i in range(1, num_nodes):
                if random.random() < 0.1:
                    agent[i] = best_agent[i]

    return best_agent, best_distance
